In [ ]:
import torch
import numpy as np
import pandas as pd
import random
import pickle
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import os

os.makedirs("visualizations", exist_ok=True)

valid_categories = [
    "امراض نسائية",
    "امراض العضلات والعظام و المفاصل",
    "امراض الجهاز الهضمي",
    "الامراض الجنسية",
    "طب الاسنان",
    "امراض القلب و الشرايين",
    "امراض العيون",
    "انف اذن وحنجرة",
    "جراحة تجميل",
    "امراض الدم"
]

def load_and_process_test_data(test_path):
    test_df = pd.read_excel(test_path)
    test_df = test_df[['q_body', 'severity', 'category']]
    test_df = test_df[test_df["category"].isin(valid_categories)]
    test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)
    test_df.to_csv("shuffled_test_data.csv", index=False)
    print("\n🔹 Test Dataset Severity Distribution (Shuffled):")
    print(test_df["severity"].value_counts())
    return test_df

def load_test_data_for_ensemble():
    test_df = pd.read_csv("shuffled_test_data.csv")
    print(f"\n✅ Loaded shuffled test dataset with {len(test_df)} samples.")
    return test_df

class CustomModel(torch.nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomModel, self).__init__()
        self.base_model = base_model 
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, num_labels)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)
        return {"loss": loss, "logits": logits}

def load_complete_model(model_path):
    base_model = AutoModel.from_pretrained(model_path)
    classifier_state = torch.load(f"{model_path}/classifier_state.pt", map_location=torch.device('cpu'))
    model = CustomModel(base_model, classifier_state['num_labels'])
    model.classifier.load_state_dict(classifier_state['classifier_state'])
    model.eval()
    return model

model_paths = [
    "/kaggle/input/ensemble_camelbert/other/default/16/bert-base-arabert-Severity",
    "/kaggle/input/ensemble_camelbert/other/default/16/BioBERT-Severity",
    "/kaggle/input/ensemble_camelbert/other/default/16/DistilBERT-Severity",
    "/kaggle/input/ensemble_camelbert/other/default/16/bert-base-multilingual-cased-Severity",
    "/kaggle/input/ensemble_camelbert/other/default/16/XLM-RoBERTa-Severity"
]
model_names = [path.split('/')[-1] for path in model_paths]

models, tokenizers, severity_mappings = [], [], []

for path in model_paths:
    tokenizers.append(AutoTokenizer.from_pretrained(path))
    model = load_complete_model(path)
    models.append(model)

    with open(f"{path}/severity_mapping.pkl", "rb") as f:
        severity_mapping = pickle.load(f)
    severity_mappings.append(severity_mapping)

if not all(severity_mappings[0] == mapping for mapping in severity_mappings):
    print("Warning: Severity mappings are different. Using the first mapping.")

severity_mapping = severity_mappings[0]
severity_mapping_reverse = {v: k for k, v in severity_mapping.items()}

def get_model_prediction(text, model, tokenizer):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    with torch.no_grad():
        outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
    logits = outputs["logits"].cpu().numpy()
    prediction = np.argmax(logits[0])
    return severity_mapping_reverse[prediction]

def get_ensemble_prediction(text, individual_predictions=None, voting='soft'):
    all_logits = []
    if individual_predictions is None:
        individual_predictions = []

    for i, (model, tokenizer) in enumerate(zip(models, tokenizers)):
        inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
        with torch.no_grad():
            outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        logits = outputs["logits"].cpu().numpy()
        all_logits.append(logits[0])
        pred = severity_mapping_reverse[np.argmax(logits[0])]
        if i >= len(individual_predictions):
            individual_predictions.append(pred)
        else:
            individual_predictions[i] = pred

    if voting == 'hard':
        predictions = [np.argmax(logits) for logits in all_logits]
        vote_counts = np.bincount(predictions, minlength=len(severity_mapping))
        ensemble_prediction = np.argmax(vote_counts)
    else:
        probs = [np.exp(logits) / np.sum(np.exp(logits)) for logits in all_logits]
        avg_probs = np.mean(probs, axis=0)
        ensemble_prediction = np.argmax(avg_probs)

    return severity_mapping_reverse[ensemble_prediction], individual_predictions

def apply_ensemble_on_test_data(print_interval=500):
    test_df = load_test_data_for_ensemble()

    results_columns = [
        "Text", 
        "True_Severity"
    ] + [f"{name}_Prediction" for name in model_names] + [
        "Ensemble_Prediction",
        "Correct"
    ]
    
    results = []
    total_samples = len(test_df)

    print(f"\n🔹 Running ensemble and individual model predictions on test dataset ({total_samples} samples)...")

    for idx, row in test_df.iterrows():
        text, true_severity = row["q_body"], row["severity"]
        individual_predictions = []
        ensemble_prediction, individual_predictions = get_ensemble_prediction(text, individual_predictions)
        
        result_row = {
            "Text": text,
            "True_Severity": true_severity
        }
        for i, model_name in enumerate(model_names):
            result_row[f"{model_name}_Prediction"] = individual_predictions[i]
        result_row["Ensemble_Prediction"] = ensemble_prediction
        result_row["Correct"] = true_severity == ensemble_prediction
        results.append(result_row)

        if (idx + 1) % print_interval == 0 or idx == total_samples - 1:
            print("\n" + "="*80)
            print(f"Processing sample {idx + 1}/{total_samples}")
            print("="*80)
            print(f"Input text: {text}\n")
            print("Individual model predictions:")
            for i, model_name in enumerate(model_names):
                print(f"- {model_name}: {individual_predictions[i]}")
            print("\nEnsemble prediction (soft voting):", ensemble_prediction)
            print("True severity:", true_severity)
            print("Correct:", true_severity == ensemble_prediction)
            print("="*80)

    results_df = pd.DataFrame(results)
    results_df.to_csv("ensemble_individual_prediction_severity.csv", index=False)
    print("\n✅ All severity predictions saved to 'ensemble_individual_prediction_severity.csv'.")

    accuracy_results = {
        "Model": ["Ensemble"] + model_names,
        "Accuracy": [
            (results_df["True_Severity"] == results_df["Ensemble_Prediction"]).mean()
        ]
    }
    for model_name in model_names:
        accuracy = (results_df["True_Severity"] == results_df[f"{model_name}_Prediction"]).mean()
        accuracy_results["Accuracy"].append(accuracy)
    
    accuracy_df = pd.DataFrame(accuracy_results)
    accuracy_df.to_csv("model_accuracies_severity.csv", index=False)
    print("\n✅ Model accuracy results saved to 'model_accuracies_severity.csv'.")

    # Print sample results
    print("\n🔹 Sample Predictions:")
    print(results_df.head())

    # Print accuracy summary
    print("\n🔹 Model Accuracy Summary:")
    print(accuracy_df)



    generate_visualizations(results_df)

def generate_visualizations(results_df):
    """
    Generate and save visualizations for research paper
    
    Parameters:
    - results_df: DataFrame containing all predictions
    """
    print("\n🔹 Generating visualizations for research paper...")

    generate_confusion_matrices(results_df)

    generate_accuracy_comparison(results_df)

    generate_severity_performance(results_df)

    generate_model_agreement_heatmap(results_df)

    generate_ensemble_improvement_chart(results_df)

    print("\n✅ All visualizations generated and saved in 'visualizations' directory")


def generate_confusion_matrices(results_df):
    """Generate and save confusion matrices for each model and the ensemble"""
    models_to_plot = model_names + ["Ensemble"]
    num_models = len(models_to_plot)
    rows = (num_models + 1) // 2

    plt.figure(figsize=(20, 4 * rows))

    for i, model_name in enumerate(models_to_plot):
        plt.subplot(rows, 2, i + 1)

        if model_name == "Ensemble":
            y_true = results_df["True_Severity"]
            y_pred = results_df["Ensemble_Prediction"]
        else:
            y_true = results_df["True_Severity"]
            y_pred = results_df[f"{model_name}_Prediction"]

        labels = sorted(results_df["True_Severity"].unique())
        cm = confusion_matrix(y_true, y_pred, labels=labels)

        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
        plt.title(f"{model_name} - Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.xticks(rotation=45, ha="right")
        plt.yticks(rotation=45)
        plt.tight_layout()

    plt.savefig("visualizations/confusion_matrices_all.png", dpi=300, bbox_inches="tight")
    plt.close()

    for model_name in models_to_plot:
        plt.figure(figsize=(8, 6))

        if model_name == "Ensemble":
            y_true = results_df["True_Severity"]
            y_pred = results_df["Ensemble_Prediction"]
        else:
            y_true = results_df["True_Severity"]
            y_pred = results_df[f"{model_name}_Prediction"]

        labels = sorted(results_df["True_Severity"].unique())
        cm = confusion_matrix(y_true, y_pred, labels=labels)

        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
        plt.title(f"{model_name} Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.xticks(rotation=45, ha="right")
        plt.yticks(rotation=45)
        plt.tight_layout()

        plt.savefig(f"visualizations/confusion_matrix_{model_name}.png", dpi=300, bbox_inches="tight")
        plt.close()


def generate_accuracy_comparison(results_df):
    """Generate bar chart comparing accuracy of all models"""
    accuracies = []
    model_labels = []

    for model_name in model_names:
        accuracy = (results_df["True_Severity"] == results_df[f"{model_name}_Prediction"]).mean()
        accuracies.append(accuracy)
        model_labels.append(model_name)

    ensemble_accuracy = (results_df["True_Severity"] == results_df["Ensemble_Prediction"]).mean()
    accuracies.append(ensemble_accuracy)
    model_labels.append("Ensemble")

    plt.figure(figsize=(12, 8))
    bars = plt.bar(model_labels, accuracies, color=['skyblue'] * len(model_names) + ['darkblue'])

    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width() / 2., height + 0.01, f'{height:.4f}', ha='center', va='bottom', fontweight='bold')

    plt.title("Model Accuracy Comparison", fontsize=16)
    plt.ylabel("Accuracy", fontsize=14)
    plt.xlabel("Model", fontsize=14)
    plt.ylim(0, max(accuracies) * 1.15)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    plt.savefig("visualizations/accuracy_comparison.png", dpi=300, bbox_inches="tight")
    plt.close()


def generate_severity_performance(results_df):
    """Generate per-severity performance chart for all models"""
    performance = {}

    valid_severities = sorted(results_df["True_Severity"].unique())
    for severity in valid_severities:
        performance[severity] = {}

    all_models = model_names + ["Ensemble"]
    for model_name in all_models:
        for severity in valid_severities:
            mask = results_df["True_Severity"] == severity
            if model_name == "Ensemble":
                correct = results_df.loc[mask, "Ensemble_Prediction"] == severity
            else:
                correct = results_df.loc[mask, f"{model_name}_Prediction"] == severity

            accuracy = correct.sum() / sum(mask) if sum(mask) > 0 else 0
            performance[severity][model_name] = accuracy

    severity_df = pd.DataFrame(performance).T

    severity_df.plot(kind='bar', figsize=(15, 10))
    plt.title("Model Performance by Severity", fontsize=16)
    plt.xlabel("Severity", fontsize=14)
    plt.ylabel("Accuracy", fontsize=14)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend(title="Model", loc='upper left', bbox_to_anchor=(1, 1))
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()

    plt.savefig("visualizations/severity_performance.png", dpi=300, bbox_inches="tight")
    plt.close()
    severity_df.to_csv("visualizations/severity_performance.csv")


def generate_model_agreement_heatmap(results_df):
    """Generate heatmap showing agreement between models"""
    agreement_matrix = {}

    all_models = model_names + ["Ensemble"]
    for model1 in all_models:
        agreement_matrix[model1] = {}
        for model2 in all_models:
            pred1 = results_df[f"{model1}_Prediction"] if model1 != "Ensemble" else results_df["Ensemble_Prediction"]
            pred2 = results_df[f"{model2}_Prediction"] if model2 != "Ensemble" else results_df["Ensemble_Prediction"]
            agreement_matrix[model1][model2] = np.mean(pred1 == pred2)

    agreement_df = pd.DataFrame(agreement_matrix)

    plt.figure(figsize=(12, 10))
    sns.heatmap(agreement_df, annot=True, fmt=".3f", cmap="YlGnBu", vmin=0, vmax=1)
    plt.title("Model Agreement Heatmap", fontsize=16)
    plt.tight_layout()

    plt.savefig("visualizations/model_agreement.png", dpi=300, bbox_inches="tight")
    plt.close()


def generate_ensemble_improvement_chart(results_df):
    """Generate chart showing where ensemble improves over individual models"""
    improvements = {}

    for model_name in model_names:
        ensemble_correct = results_df["True_Severity"] == results_df["Ensemble_Prediction"]
        model_wrong = results_df["True_Severity"] != results_df[f"{model_name}_Prediction"]
        improvements[model_name] = (ensemble_correct & model_wrong).sum()

    plt.figure(figsize=(12, 6))
    plt.bar(improvements.keys(), improvements.values(), color='green')
    plt.title("Cases Where Ensemble Corrects Individual Model Errors", fontsize=16)
    plt.ylabel("Number of Improvements", fontsize=14)
    plt.xlabel("Base Model", fontsize=14)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(rotation=45, ha="right")

    for i, (model, count) in enumerate(improvements.items()):
        plt.text(i, count + 2, str(count), ha='center', fontweight='bold')

    plt.tight_layout()
    plt.savefig("visualizations/ensemble_improvements.png", dpi=300, bbox_inches="tight")
    plt.close()

    total_samples = len(results_df)
    improvement_df = pd.DataFrame({
        'Model': list(improvements.keys()),
        'Improvement_Count': list(improvements.values()),
        'Improvement_Percentage': [count / total_samples * 100 for count in improvements.values()]
    })
    improvement_df.to_csv("visualizations/ensemble_improvement_stats.csv", index=False)

if __name__ == "__main__":
    test_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Test.xlsx"

    test_df = load_and_process_test_data(test_path)

    apply_ensemble_on_test_data(print_interval=500)

2025-06-18 16:05:26.674478: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750262726.697158     118 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750262726.703921     118 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered



🔹 Test Dataset Severity Distribution (Shuffled):
severity
حرج        30717
غير حرج     9072
Name: count, dtype: int64

✅ Loaded shuffled test dataset with 39789 samples.

🔹 Running ensemble and individual model predictions on test dataset (39789 samples)...

Processing sample 500/39789
Input text: اريد التاكد من عذريه زوجتي والتاكد من ان لم يسبق ممارسه الجنس اريد جواب مفصل

Individual model predictions:
- bert-base-arabert-Severity: حرج
- BioBERT-Severity: حرج
- DistilBERT-Severity: حرج
- bert-base-multilingual-cased-Severity: حرج
- XLM-RoBERTa-Severity: حرج

Ensemble prediction (soft voting): حرج
True severity: حرج
Correct: True

Processing sample 1000/39789
Input text: انا فتاه عمري 17 سنه مذ ان كان عمري 12 سنه وانا استعمل اقراص بونستيل لان الالم شديد وسمعت ان هذه الاقراص تتسبب في العقم هل هذا صحيح

Individual model predictions:
- bert-base-arabert-Severity: حرج
- BioBERT-Severity: حرج
- DistilBERT-Severity: حرج
- bert-base-multilingual-cased-Severity: حرج
- XLM-RoBERTa-Severity: غي

/tmp/ipykernel_118/2331064987.py:266: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_118/2331064987.py:266: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_118/2331064987.py:266: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_118/2331064987.py:266: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/tmp/ipykernel_118/2331064987.py:266: UserWarning: The figure layout has changed to tight
  plt.tight_layout()



✅ All visualizations generated and saved in 'visualizations' directory


In [ ]:
import os
import zipfile
import IPython.display
from datetime import datetime

def create_zip_and_download_link():
    """
    Compresses all generated files and folders into a single zip file
    and creates a download link.
    
    Returns:
        IPython.display.HTML: HTML download link for the zip file
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_filename = f"ensemble_results_{timestamp}.zip"
    
    items_to_zip = [
        "visualizations",  
        "ensemble_individual_prediction.csv",
        "model_accuracies.csv",
        "shuffled_test_data.csv"
    ]
    
    print(f"🔹 Creating zip file: {zip_filename}")
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for item in items_to_zip:
            if os.path.isfile(item):
                zipf.write(item)
                print(f"  ✅ Added file: {item}")
            elif os.path.isdir(item):
                for root, _, files in os.walk(item):
                    for file in files:
                        file_path = os.path.join(root, file)
                        
                        zipf.write(file_path)
                print(f"  ✅ Added directory: {item} and its contents")
            else:
                print(f"  ❌ Item not found: {item}")
    
    file_size = os.path.getsize(zip_filename) / (1024 * 1024)  # Size in MB
    
    print(f"✅ Zip file created successfully: {zip_filename} ({file_size:.2f} MB)")
    
    if 'google.colab' in str(get_ipython()):
        from google.colab import files
        print("📥 Starting download...")
        files.download(zip_filename)
        return f"Download initiated for {zip_filename}"
    else:
        html_code = f'''
        <a href="./files/{zip_filename}" download="{zip_filename}" 
           style="display: inline-block; padding: 10px 15px; 
                  background-color: #4CAF50; color: white; 
                  text-align: center; text-decoration: none; 
                  font-size: 16px; border-radius: 5px;">
           Download Results ({file_size:.2f} MB)
        </a>
        '''
        return IPython.display.HTML(html_code)

download_link = create_zip_and_download_link()
display(download_link)

🔹 Creating zip file: ensemble_results_20250618_234008.zip
  ✅ Added directory: visualizations and its contents
  ❌ Item not found: ensemble_individual_prediction.csv
  ❌ Item not found: model_accuracies.csv
  ✅ Added file: shuffled_test_data.csv
✅ Zip file created successfully: ensemble_results_20250618_234008.zip (3.74 MB)


In [ ]:
import os
import zipfile
import IPython.display
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import arabic_reshaper
from bidi.algorithm import get_display

def setup_arabic_plots():
    """Configure matplotlib to properly display Arabic text"""
    plt.rcParams['font.family'] = 'Arial'
    plt.rcParams['font.size'] = 12
    
    print("✅ Matplotlib configured for Arabic text display")

def reshape_arabic_text(text):
    """
    Reshape Arabic text for proper display in matplotlib
    
    Args:
        text (str): Arabic text to reshape
        
    Returns:
        str: Reshaped and reordered text ready for display
    """
    if isinstance(text, str):
        reshaped_text = arabic_reshaper.reshape(text)
        bidi_text = get_display(reshaped_text)
        return bidi_text
    return text

def generate_confusion_matrices_arabic(results_df, valid_categories):
    """
    Generate confusion matrices with correctly displayed Arabic labels
    
    Args:
        results_df (pd.DataFrame): DataFrame with predictions
        valid_categories (list): List of category names in Arabic
    """
    setup_arabic_plots()
    
    reshaped_categories = [reshape_arabic_text(cat) for cat in valid_categories]
    
    plt.figure(figsize=(20, 16))
    
    models_to_plot = model_names + ["Ensemble"]
    num_models = len(models_to_plot)
    rows = (num_models + 1) // 2
    
    for i, model_name in enumerate(models_to_plot):
        plt.subplot(rows, 2, i+1)
        
        if model_name == "Ensemble":
            y_true = results_df["True_Category"]
            y_pred = results_df["Ensemble_Prediction"]
        else:
            y_true = results_df["True_Category"]
            y_pred = results_df[f"{model_name}_Prediction"]
            
        cm = confusion_matrix(y_true, y_pred, labels=valid_categories)
        
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
                   xticklabels=reshaped_categories, 
                   yticklabels=reshaped_categories)
        plt.title(f"{model_name} Confusion Matrix")
        plt.xlabel("Predicted Category")
        plt.ylabel("True Category")
        plt.xticks(rotation=45, ha="right")
        plt.yticks(rotation=45)
        plt.tight_layout()
    
    plt.savefig("visualizations/confusion_matrices.png", dpi=300, bbox_inches="tight")
    plt.close()
    
    for model_name in models_to_plot:
        plt.figure(figsize=(12, 10))
        
        if model_name == "Ensemble":
            y_true = results_df["True_Category"]
            y_pred = results_df["Ensemble_Prediction"]
        else:
            y_true = results_df["True_Category"]
            y_pred = results_df[f"{model_name}_Prediction"]
            
        cm = confusion_matrix(y_true, y_pred, labels=valid_categories)
        
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", 
                   xticklabels=reshaped_categories, 
                   yticklabels=reshaped_categories)
        plt.title(f"{model_name} Confusion Matrix")
        plt.xlabel("Predicted Category")
        plt.ylabel("True Category")
        plt.xticks(rotation=45, ha="right")
        plt.yticks(rotation=45)
        plt.tight_layout()
        
        plt.savefig(f"visualizations/confusion_matrix_{model_name}.png", dpi=300, bbox_inches="tight")
        plt.close()

def reshape_dataframe_arabic(df):
    """
    Reshape Arabic text in DataFrames for proper display
    
    Args:
        df (pd.DataFrame): DataFrame containing Arabic text
        
    Returns:
        pd.DataFrame: DataFrame with reshaped Arabic text
    """
    reshaped_df = df.copy()
    
    for col in reshaped_df.columns:
        if reshaped_df[col].dtype == 'object':
            reshaped_df[col] = reshaped_df[col].apply(reshape_arabic_text)
    
    return reshaped_df

def create_zip_with_arabic_support():
    """
    Compresses all generated files and folders into a zip file and creates a download link.
    Ensures Arabic text is properly reshaped in visualizations and CSV files.
    
    Returns:
        IPython.display.HTML: HTML download link for the zip file
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    zip_filename = f"arabic_ensemble_results_{timestamp}.zip"
    
    # First, ensure all CSV files have properly reshaped Arabic text
    csv_files = [
        "ensemble_individual_prediction.csv", 
        "model_accuracies.csv",
        "shuffled_test_data.csv"
    ]
    
    print("🔹 Reshaping Arabic text in CSV files...")
    for csv_file in csv_files:
        if os.path.exists(csv_file):
            try:
                # Read the CSV
                df = pd.read_csv(csv_file)
                # Reshape Arabic text
                reshaped_df = reshape_dataframe_arabic(df)
                # Save with a new name
                reshaped_file = f"reshaped_{csv_file}"
                reshaped_df.to_csv(reshaped_file, index=False, encoding='utf-8-sig')
                print(f"  ✅ Reshaped Arabic text in: {csv_file}")
            except Exception as e:
                print(f"  ❌ Error reshaping {csv_file}: {e}")
    
    items_to_zip = [
        "visualizations",  
        "reshaped_ensemble_individual_prediction.csv",
        "reshaped_model_accuracies.csv",
        "reshaped_shuffled_test_data.csv",
        "ensemble_individual_prediction.csv", 
        "model_accuracies.csv",
        "shuffled_test_data.csv"
    ]
    
    print(f"🔹 Creating zip file: {zip_filename}")
    
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for item in items_to_zip:
            if os.path.isfile(item):
                zipf.write(item)
                print(f"  ✅ Added file: {item}")
            elif os.path.isdir(item):
                for root, _, files in os.walk(item):
                    for file in files:
                        file_path = os.path.join(root, file)
                        arcname = os.path.relpath(file_path)
                        zipf.write(file_path, arcname)
                print(f"  ✅ Added directory: {item} and its contents")
            else:
                print(f"  ⚠️ Item not found (skipped): {item}")
    
    file_size = os.path.getsize(zip_filename) / (1024 * 1024) 
    
    print(f"✅ Zip file created successfully: {zip_filename} ({file_size:.2f} MB)")
    
    if 'google.colab' in str(get_ipython()):
        # For Google Colab
        from google.colab import files
        print("📥 Starting download...")
        files.download(zip_filename)
        return f"Download initiated for {zip_filename}"
    
    elif 'kaggle_web_client' in str(get_ipython()):
        # For Kaggle
        try:
            # Make sure the file is in the output directory
            from kaggle.api.kaggle_api_extended import KaggleApi
            api = KaggleApi()
            api.authenticate()
            
            print(f"✅ File available for download in Kaggle Output tab: {zip_filename}")
            
            html_code = f'''
            <div style="margin: 20px 0; padding: 15px; border: 1px solid #ddd; border-radius: 5px; background-color: #f9f9f9;">
                <h3 style="margin-top: 0; color: #333;">Download Instructions</h3>
                <p>Your file <strong>{zip_filename}</strong> ({file_size:.2f} MB) has been created.</p>
                <p>To download this file:</p>
                <ol>
                    <li>Go to the <strong>Output</strong> tab above</li>
                    <li>Find <strong>{zip_filename}</strong></li>
                    <li>Click the download button</li>
                </ol>
            </div>
            '''
            return IPython.display.HTML(html_code)
        
        except Exception as e:
            print(f"Warning: Could not authenticate with Kaggle API: {e}")
            # Fallback
            html_code = f'''
            <div style="margin: 20px 0;">
                <p>File created: {zip_filename} ({file_size:.2f} MB)</p>
                <p>To download, go to the "Output" tab in this Kaggle notebook.</p>
            </div>
            '''
            return IPython.display.HTML(html_code)
    
    else:
        # For standard Jupyter environment
        html_code = f'''
        <a href="{zip_filename}" download="{zip_filename}" 
           style="display: inline-block; padding: 10px 15px; 
                  background-color: #4CAF50; color: white; 
                  text-align: center; text-decoration: none; 
                  font-size: 16px; border-radius: 5px;">
           Download Results ({file_size:.2f} MB)
        </a>
        '''
        return IPython.display.HTML(html_code)

# Example usage - add this to your main function
if __name__ == "__main__":
    # Make sure valid_categories is available
    valid_categories = [
        "امراض نسائية",
        "امراض العضلات والعظام و المفاصل",
        "امراض الجهاز الهضمي",
        "الامراض الجنسية",
        "طب الاسنان",
        "امراض القلب و الشرايين",
        "امراض العيون",
        "انف اذن وحنجرة",
        "جراحة تجميل",
        "امراض الدم"
    ]
    
    # Install required packages if not already installed
    try:
        import arabic_reshaper
        import bidi
    except ImportError:
        print("🔹 Installing Arabic reshaper packages...")
        !pip install arabic-reshaper python-bidi
        import arabic_reshaper
        from bidi.algorithm import get_display
        print("✅ Arabic reshaper packages installed")
    
    # Assuming you've already generated your results_df and visualizations
    # Regenerate confusion matrices with Arabic reshaping
    # Uncomment and modify as needed:
    # results_df = pd.read_csv("ensemble_individual_prediction.csv")
    # generate_confusion_matrices_arabic(results_df, valid_categories)
    
    # Create the zip file with Arabic support and get download link
    download_link = create_zip_with_arabic_support()
    display(download_link)

🔹 Reshaping Arabic text in CSV files...
  ✅ Reshaped Arabic text in: shuffled_test_data.csv
🔹 Creating zip file: arabic_ensemble_results_20250618_234753.zip
  ✅ Added directory: visualizations and its contents
  ⚠️ Item not found (skipped): reshaped_ensemble_individual_prediction.csv
  ⚠️ Item not found (skipped): reshaped_model_accuracies.csv
  ✅ Added file: reshaped_shuffled_test_data.csv
  ⚠️ Item not found (skipped): ensemble_individual_prediction.csv
  ⚠️ Item not found (skipped): model_accuracies.csv
  ✅ Added file: shuffled_test_data.csv
✅ Zip file created successfully: arabic_ensemble_results_20250618_234753.zip (6.36 MB)


In [5]:
pip install arabic-reshaper


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
import numpy as np
import pandas as pd
import pickle
from transformers import AutoTokenizer, AutoModel
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import os

# Create visualization folder
os.makedirs("visualizations", exist_ok=True)

# 🔹 Valid categories
valid_categories = [
    "امراض نسائية", "امراض العضلات والعظام و المفاصل", "امراض الجهاز الهضمي",
    "الامراض الجنسية", "طب الاسنان", "امراض القلب و الشرايين", "امراض العيون",
    "انف اذن وحنجرة", "جراحة تجميل", "امراض الدم"
]

def load_and_process_test_data(test_path):
    test_df = pd.read_excel(test_path)
    test_df = test_df[['q_body', 'severity', 'category']]
    test_df = test_df[test_df["category"].isin(valid_categories)]
    test_df = test_df.sample(frac=1, random_state=42).reset_index(drop=True)
    test_df.to_csv("shuffled_test_data.csv", index=False)
    print("\n🔹 Test Dataset Severity Distribution (Shuffled):")
    print(test_df["severity"].value_counts())
    return test_df

def load_test_data_for_ensemble():
    test_df = pd.read_csv("shuffled_test_data.csv")
    print(f"\n✅ Loaded shuffled test dataset with {len(test_df)} samples.")
    return test_df

class CustomModel(torch.nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomModel, self).__init__()
        self.base_model = base_model
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(base_model.config.hidden_size, num_labels)
        self.loss_fn = torch.nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        loss = self.loss_fn(logits, labels) if labels is not None else None
        return {"loss": loss, "logits": logits}

model_paths = [
    "/kaggle/input/ensemblee/other/default/1/AraBert",
    "/kaggle/input/ensemblee/other/default/1/BioBert (2)",
    "/kaggle/input/ensemblee/other/default/1/distilBert",
    "/kaggle/input/ensemblee/other/default/1/multiBert",
    "/kaggle/input/ensemblee/other/default/1/xlmRoBERTaa"
]
model_names = [path.split('/')[-1] for path in model_paths]
model_weights = [0.3, 0.25, 0.15, 0.15, 0.15]

models, tokenizers, severity_mappings = [], [], []
for path in model_paths:
    tokenizer = AutoTokenizer.from_pretrained(path)
    tokenizers.append(tokenizer)

    base_model = AutoModel.from_pretrained(path)
    classifier_state = torch.load(f"{path}/classifier_state.pt", map_location=torch.device('cpu'))
    model = CustomModel(base_model, classifier_state['num_labels'])
    model.classifier.load_state_dict(classifier_state['classifier_state'])
    model.eval()
    models.append(model)

    with open(f"{path}/severity_mapping.pkl", "rb") as f:
        severity_mappings.append(pickle.load(f))

severity_mapping = severity_mappings[0]
severity_mapping_reverse = {v: k for k, v in severity_mapping.items()}

def get_ensemble_prediction(text, individual_predictions=None):
    all_logits = []
    if individual_predictions is None:
        individual_predictions = []

    for i, (model, tokenizer) in enumerate(zip(models, tokenizers)):
        inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
        with torch.no_grad():
            outputs = model(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
        logits = outputs["logits"].cpu().numpy()
        all_logits.append(logits[0])
        pred = severity_mapping_reverse[np.argmax(logits[0])]
        if i >= len(individual_predictions):
            individual_predictions.append(pred)
        else:
            individual_predictions[i] = pred

    probs = [np.exp(logits) / np.sum(np.exp(logits)) for logits in all_logits]
    weighted_probs = np.sum([w * p for w, p in zip(model_weights, probs)], axis=0)
    ensemble_prediction = np.argmax(weighted_probs)

    return severity_mapping_reverse[ensemble_prediction], individual_predictions

def apply_ensemble_on_test_data(print_interval=500):
    test_df = load_test_data_for_ensemble()

    results = []
    for idx, row in test_df.iterrows():
        text, true_severity = row["q_body"], row["severity"]
        individual_predictions = []
        ensemble_prediction, individual_predictions = get_ensemble_prediction(text, individual_predictions)

        result_row = {
            "Text": text,
            "True_Severity": true_severity,
            "Ensemble_Prediction": ensemble_prediction,
            "Correct": true_severity == ensemble_prediction
        }
        for i, model_name in enumerate(model_names):
            result_row[f"{model_name}_Prediction"] = individual_predictions[i]
        results.append(result_row)

        if (idx + 1) % print_interval == 0 or idx == len(test_df) - 1:
            print(f"Processed {idx + 1}/{len(test_df)}")

    results_df = pd.DataFrame(results)
    results_df.to_csv("ensemble_individual_prediction_severity.csv", index=False)

    acc_data = {
        "Model": ["Ensemble"] + model_names,
        "Accuracy": [(results_df["True_Severity"] == results_df["Ensemble_Prediction"]).mean()]
    }
    for model_name in model_names:
        acc = (results_df["True_Severity"] == results_df[f"{model_name}_Prediction"]).mean()
        acc_data["Accuracy"].append(acc)

    accuracy_df = pd.DataFrame(acc_data)
    accuracy_df.to_csv("model_accuracies_severity.csv", index=False)

    generate_visualizations(results_df)

# 🔹 Visualization Functions
def generate_visualizations(results_df):
    print("\n🔹 Generating visualizations...")
    generate_confusion_matrices(results_df)
    generate_accuracy_comparison(results_df)
    generate_severity_performance(results_df)
    generate_model_agreement_heatmap(results_df)
    generate_ensemble_improvement_chart(results_df)
    print("\n✅ All visualizations saved.")

def generate_confusion_matrices(results_df):
    models_to_plot = model_names + ["Ensemble"]
    rows = (len(models_to_plot) + 1) // 2
    plt.figure(figsize=(20, 4 * rows))

    for i, model_name in enumerate(models_to_plot):
        plt.subplot(rows, 2, i + 1)
        y_true = results_df["True_Severity"]
        y_pred = results_df["Ensemble_Prediction"] if model_name == "Ensemble" else results_df[f"{model_name}_Prediction"]
        labels = sorted(results_df["True_Severity"].unique())
        cm = confusion_matrix(y_true, y_pred, labels=labels)
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
        plt.title(f"{model_name} - Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.xticks(rotation=45)
        plt.tight_layout()

    plt.savefig("visualizations/confusion_matrices_all.png", dpi=300, bbox_inches="tight")
    plt.close()

def generate_accuracy_comparison(results_df):
    accuracies = []
    model_labels = []
    for model_name in model_names:
        acc = (results_df["True_Severity"] == results_df[f"{model_name}_Prediction"]).mean()
        accuracies.append(acc)
        model_labels.append(model_name)
    ensemble_acc = (results_df["True_Severity"] == results_df["Ensemble_Prediction"]).mean()
    accuracies.append(ensemble_acc)
    model_labels.append("Ensemble")

    plt.figure(figsize=(12, 8))
    bars = plt.bar(model_labels, accuracies, color=['skyblue'] * len(model_names) + ['darkblue'])
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height + 0.01, f'{height:.4f}', ha='center', va='bottom')
    plt.title("Model Accuracy Comparison")
    plt.ylabel("Accuracy")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("visualizations/accuracy_comparison.png", dpi=300)
    plt.close()

def generate_severity_performance(results_df):
    performance = {}
    valid_severities = sorted(results_df["True_Severity"].unique())
    for s in valid_severities:
        performance[s] = {}

    for model_name in model_names + ["Ensemble"]:
        for s in valid_severities:
            mask = results_df["True_Severity"] == s
            pred = results_df["Ensemble_Prediction"] if model_name == "Ensemble" else results_df[f"{model_name}_Prediction"]
            acc = (pred[mask] == s).mean()
            performance[s][model_name] = acc

    df = pd.DataFrame(performance).T
    df.plot(kind="bar", figsize=(15, 10))
    plt.title("Model Performance by Severity")
    plt.xlabel("Severity")
    plt.ylabel("Accuracy")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig("visualizations/severity_performance.png", dpi=300)
    plt.close()
    df.to_csv("visualizations/severity_performance.csv")

def generate_model_agreement_heatmap(results_df):
    agreement = {}
    all_models = model_names + ["Ensemble"]
    for m1 in all_models:
        agreement[m1] = {}
        for m2 in all_models:
            p1 = results_df[f"{m1}_Prediction"] if m1 != "Ensemble" else results_df["Ensemble_Prediction"]
            p2 = results_df[f"{m2}_Prediction"] if m2 != "Ensemble" else results_df["Ensemble_Prediction"]
            agreement[m1][m2] = (p1 == p2).mean()

    df = pd.DataFrame(agreement)
    plt.figure(figsize=(12, 10))
    sns.heatmap(df, annot=True, fmt=".2f", cmap="YlGnBu", vmin=0, vmax=1)
    plt.title("Model Agreement Heatmap")
    plt.tight_layout()
    plt.savefig("visualizations/model_agreement.png", dpi=300)
    plt.close()

def generate_ensemble_improvement_chart(results_df):
    improvements = {}
    for model_name in model_names:
        ensemble_correct = results_df["True_Severity"] == results_df["Ensemble_Prediction"]
        model_wrong = results_df["True_Severity"] != results_df[f"{model_name}_Prediction"]
        improvements[model_name] = (ensemble_correct & model_wrong).sum()

    plt.figure(figsize=(12, 6))
    plt.bar(improvements.keys(), improvements.values(), color='green')
    plt.title("Ensemble Improvements Over Base Models")
    plt.xlabel("Model")
    plt.ylabel("Improved Cases")
    for i, (model, val) in enumerate(improvements.items()):
        plt.text(i, val + 1, str(val), ha='center')
    plt.tight_layout()
    plt.savefig("visualizations/ensemble_improvements.png", dpi=300)
    plt.close()

    total = len(results_df)
    pd.DataFrame({
        "Model": list(improvements.keys()),
        "Improvement_Count": list(improvements.values()),
        "Improvement_Percentage": [v / total * 100 for v in improvements.values()]
    }).to_csv("visualizations/ensemble_improvement_stats.csv", index=False)

# 🔹 Run the pipeline
if __name__ == "__main__":
    test_path = "/kaggle/input/maqa-unbalanced-with-severity/MAQA_Severity_Test.xlsx"
    load_and_process_test_data(test_path)
    apply_ensemble_on_test_data(print_interval=500)
